# Instagram Fake Account Detection

Experiments on 28 June 2023

In [1]:
from dataset.normalizer import json_importer_full, csv_importer_full
from dataset.utils import find_demarcator
from utils.utils import experiment

/home/alberto/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-06-28 18:22:04.195504: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-28 18:22:04.233950: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:7630] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-06-28 18:22:04.233986: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-06-28 18:22:04.234005: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS w

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Import `Instagram Fake and Automated Account Detection` dataset

In [2]:
fake_spz = json_importer_full("./dataset/sources/automatedAccountData.json", True)
correct_spz = json_importer_full("./dataset/sources/nonautomatedAccountData.json", False)
default_dataset = csv_importer_full("./dataset/sources/user_fake_authentic_2class.csv")

Now loading from file ./dataset/sources/automatedAccountData.json...
Loaded 700 entries from source ./dataset/sources/automatedAccountData.json
Now loading from file ./dataset/sources/nonautomatedAccountData.json...
Loaded 700 entries from source ./dataset/sources/nonautomatedAccountData.json
Now loading from file ./dataset/sources/user_fake_authentic_2class.csv...
Loaded 65327 entries from source ./dataset/sources/user_fake_authentic_2class.csv


Import `IJECE` dataset

In [3]:
idx = find_demarcator(default_dataset)
fake_IJECE = default_dataset[:idx]
correct_IJECE = default_dataset[idx:]

- dt - Decision Tree
- lr - Linear Regression
- nb - Naive Bayes
- rf - Random Forest
- dl - Percettrone Multistrato

In [4]:
exp = "dl"
n_iter = 1

results = {"spz": dict(), "IJECE": dict()}

print("\nRunning test on dataset 'Instagram Fake and Automated Account Detection' (internal name: 'spz')...")
res = experiment(fake_spz, correct_spz, csv=False, mode=exp, n_iter=n_iter)
results["spz"][exp] = res

print("Running test on dataset 'IJECE' (internal name: 'IJECE')...")
res = experiment(fake_IJECE, correct_IJECE, csv=True, mode=exp, n_iter=n_iter)
results["IJECE"][exp] = res


Running test on dataset 'Instagram Fake and Automated Account Detection' (internal name: 'spz')...
Calculating metrics for Deep Learning over 1 times
Epoch 1/100fault model 1/1      
978/978 [==============================] - 2s 1ms/step - loss: 498.2408 - accuracy: 0.4928 - precision: 0.8023 - recall: 0.8630
Epoch 2/100
978/978 [==============================] - 1s 1ms/step - loss: 367.2503 - accuracy: 0.5358 - precision: 0.7836 - recall: 0.8221
Epoch 3/100
978/978 [==============================] - 1s 1ms/step - loss: 217.8700 - accuracy: 0.5184 - precision: 0.8178 - recall: 0.8630
Epoch 4/100
978/978 [==============================] - 1s 1ms/step - loss: 96.1797 - accuracy: 0.5961 - precision: 0.7891 - recall: 0.8875
Epoch 5/100
978/978 [==============================] - 1s 1ms/step - loss: 8.7534 - accuracy: 0.5603 - precision: 0.8537 - recall: 0.9305
Epoch 6/100
978/978 [==============================] - 1s 1ms/step - loss: 574.4297 - accuracy: 0.5337 - precision: 0.8207 - recall